In [4]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
## Removing the observations with N/As
heart = heart.dropna()

In [6]:
## Defining the input and target variables
X = heart[['age','currentSmoker','totChol','BMI','heartRate']]
Y = heart['TenYearCHD']

## SPlitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [8]:
## Building the logistic model
logit_md = LogisticRegression().fit(X_train, Y_train)

##Predicting the test dataset
logit_pred = logit_md.predict_proba(X_test)[:,1]
logit_pred

array([0.05642467, 0.0900376 , 0.22477736, 0.15260205, 0.04009915,
       0.0587168 , 0.11207704, 0.05671585, 0.26647154, 0.23770787,
       0.05709382, 0.13055466, 0.12597766, 0.08702677, 0.0690376 ,
       0.17696239, 0.05498225, 0.16553658, 0.0418935 , 0.29953953,
       0.07364956, 0.10001596, 0.06478432, 0.25487472, 0.05520256,
       0.04002731, 0.08190089, 0.10459065, 0.04231038, 0.10961855,
       0.2944258 , 0.14851834, 0.12837768, 0.18612603, 0.20409627,
       0.18747723, 0.22925175, 0.05236044, 0.19434967, 0.12391064,
       0.11134134, 0.04329512, 0.42742327, 0.13498185, 0.04596235,
       0.11479767, 0.24111694, 0.05700282, 0.05559288, 0.11992108,
       0.05209013, 0.05961336, 0.09010471, 0.16203747, 0.13996226,
       0.07081506, 0.25524276, 0.14856186, 0.07312559, 0.06104882,
       0.07612953, 0.49394144, 0.04367738, 0.3007913 , 0.10482719,
       0.13266855, 0.10629084, 0.30595862, 0.0481251 , 0.06971941,
       0.24914528, 0.28897467, 0.13740988, 0.02890459, 0.13361

In [9]:
## Changing lielihoods to labels
logit_labels = np.where(logit_pred < 0.25, 0 , 1)
logit_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
## Constructing the confusion matrix
confusion_matrix(Y_test, logit_labels)

array([[523, 107],
       [ 66,  36]])

In [11]:
accuracy_score(Y_test, logit_labels)

0.7636612021857924